In [1]:
from Utils.data import Data
import plotly.express as px
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from Utils.request import Request

In [2]:
server = Data()

In [3]:
dataframes = server.read_all_csv()

In [4]:
request = Request(dataframes)

In [5]:
order = dataframes["ODL_ORDER"]
allergy = dataframes["ODL_ALLERGY"]
orderables = dataframes["ODL_ORDERABLES"]
order_item = dataframes["ODL_ORDER_ITEM"]
allergy_customer = dataframes["ODL_ALLERGY_CUSTOMER"]
restaurant = dataframes["ODL_RESTAURANT"]


data1 = allergy
data2 = orderables

newData = pd.merge(data2,data1, on="id",how="outer")

In [6]:
newData = newData.rename(columns={"name_x":"orderables"})
newData = newData.rename(columns={"name_y":"allergy"})

newData["allergy"].fillna(value="no allergy",inplace=True)
newData["severity"].fillna(value="no severity",inplace=True)

newData["allergy"] = newData["allergy"].apply(lambda x: "fish" if x == "fishe" or x == "fishs" else x)

## Question
### Most expensive orderable

In [7]:
most_orderables = orderables[["name","price"]].sort_values("price",ascending=False).head(10)
most_orderables

,name,price
523,Chef Thomas Spicy Seafood Risotto Flavored Wit...,41.0
1329,Joes Deep Dish Apple Pie,40.9
2683,"Seafood Casserole. Scallops, Shrimp, Lobster a...",40.8
2569,Spinach Omelette WITH BACON AND CHEESE,40.4
2495,"PoBoy Sandwich. Fried Oysters, Lettuce, Tomato...",40.3
812,Fraise,39.9
398,Pacific Coast Sea Bass,39.7
801,Veal Chop with Wild Mushrooms & Potato Gnocchi,39.7
2592,Fried Fish (2) w. Beef Fried Rice,39.6
2288,GINGERED SPRING TOMATO BISQUE WITH BABY ROCK S...,39.5


## Question
### What is the least expensive dishes

In [8]:
led = newData[["orderables","price"]].sort_values("price",ascending=True).head(10)
led

,orderables,price
124,Main Courses with House Salad (balsamic vinaig...,8.2
565,Sorbet,8.2
1703,Assorted Fruit Topped Danish ...1.60,8.2
20,French fries,8.2
1140,Moo Shu Baby Shrimp w. 4 Pancakes,8.2
2796,"Crozes-Hermitage, Vidal Fleury, 1994",8.3
1055,"Pouilly Fuisse, Les Chailloux, Philippe Forset...",8.4
2418,"Cabernet Sauvignon, Bodega Weinert, 1995. (Arg...",8.5
165,"GRAVLAX SALMON, DILL-MARINATED",8.5
2612,Brie,8.5


In [9]:
led.index[0]

124

## Question 
### Dishes that contains the most allergies

In [10]:
newData[["allergy","orderables"]].groupby("orderables").count().sort_values("allergy",ascending=False).head(10)

,allergy
orderables,
Onion Soup GratinÃƒÂ©e,8
"Mango Carnitas Enchiladas, slowly simmered lean pork loin, onions, and spices layered between two blue corn tortillas with mango chutney, salsa, sour cream, and white cheddar cheese smothered with green chile and garnished with sour cream and corn salsa served with pinto beans and Basmati rice",8
Braised Pork Loin stuffed with sweet sausage and served with artichoke and zucchini,7
"Pan Fried Scallops with Anchovy, Capers & Lemon",6
Sanka,6
"Wraps: W5 Vegetarian Grilled Eggplant, Squash & Zucchini with Avocado, Sprouts & Humus ... 5.75",6
Chocolate Raspberry Cake,6
Beefeater Gin,6
"Cabernet Sauvignon, Duckhorn, 1986. (Napa Valley)",6


## Question
### The customers with the most order

In [11]:
number_order = order.groupby("customer_id").count()
cmo = number_order.sort_values("creation_date",ascending=False).head(10)
cmo

,restaurant_id,creation_date,id
customer_id,,,
331,31,31,31
547,29,29,29
2136,28,28,28
148,28,28,28
2735,27,27,27
4074,26,26,26
1131,26,26,26
733,25,25,25
6397,25,25,25


## Question
### The customer with the most order

In [12]:
print("customer " + str(cmo.index[0]))

customer 331


## Question
### What are the most ordered dishes

In [13]:
data1 = order
data2 = orderables
newData = pd.merge(data2,data1,how="outer",on="restaurant_id")
cmo = newData.groupby("name").count().sort_values("id_y",ascending=False)
cmo

,price,restaurant_id,id_x,creation_date,customer_id,id_y
name,,,,,,
Shrimp Cocktail,3442,3442,3442,3442,3442,3442
Braised Pork Loin stuffed with sweet sausage and served with artichoke and zucchini,2876,2876,2876,2876,2876,2876
"Wraps: W6 Italian Tuna Wrap (No Mayo No Oil), Plum Tomato, Leaf Lettuce & Sprouts ... 5.95",2799,2799,2799,2799,2799,2799
Unfiltered Apple Juice,2720,2720,2720,2720,2720,2720
Terrine de canard,2668,2668,2668,2668,2668,2668
...,...,...,...,...,...,...
CAVIAR with Chopped Egg,8,8,8,8,8,8
Fresh Squeezed Orange Juice,8,8,8,8,8,8
Fruit Platter: Sliced Fruit Served with your Choice of Cottage Cheese or Low Fat Yogurt with Tuna Salad,5,5,5,5,5,5


In [14]:
cmo.drop("id_x",inplace=True,axis=1)
cmo.drop("id_y",inplace=True,axis=1)

In [15]:
cmo

,price,restaurant_id,creation_date,customer_id
name,,,,
Shrimp Cocktail,3442,3442,3442,3442
Braised Pork Loin stuffed with sweet sausage and served with artichoke and zucchini,2876,2876,2876,2876
"Wraps: W6 Italian Tuna Wrap (No Mayo No Oil), Plum Tomato, Leaf Lettuce & Sprouts ... 5.95",2799,2799,2799,2799
Unfiltered Apple Juice,2720,2720,2720,2720
Terrine de canard,2668,2668,2668,2668
...,...,...,...,...
CAVIAR with Chopped Egg,8,8,8,8
Fresh Squeezed Orange Juice,8,8,8,8
Fruit Platter: Sliced Fruit Served with your Choice of Cottage Cheese or Low Fat Yogurt with Tuna Salad,5,5,5,5


## Question
### What is the most ordered dish

In [16]:
cmo.index[0]

'Shrimp Cocktail'

## Question
